In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily

# Set time period
start = datetime(2010, 1, 1)
end = datetime.now() - timedelta(days=1)  # Automatically set to yesterday
vancouver = Point(49.2497, -123.1193, 70)
# Create Point for Hanoi, Vietnam
hanoi = Point(21.0285, 105.8542, 15)
london = Point(51.5074, -0.1278, 11)
# Get daily data
df = Daily(london, start, end)
df = df.fetch()




In [46]:
df

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2010-01-01,1.6,0.2,3.2,0.0,NaN,324.0,11.8,NaN,1006.1,NaN
2010-01-02,2.7,0.5,5.4,0.0,NaN,254.0,15.0,NaN,1016.0,NaN
2010-01-03,1.9,0.1,3.2,0.0,NaN,43.0,6.4,NaN,1023.6,NaN
2010-01-04,-0.3,-2.0,1.9,0.0,NaN,280.0,8.9,NaN,1020.0,NaN
2010-01-05,1.0,-1.8,3.2,0.3,20.0,178.0,9.8,NaN,1005.9,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-11-26,5.8,4.6,7.8,0.3,NaN,209.0,9.3,24.1,1015.1,NaN
2024-11-27,5.2,1.2,8.3,16.4,NaN,347.0,15.1,38.9,1013.3,NaN
2024-11-28,1.7,-1.0,4.5,0.0,NaN,78.0,6.5,20.4,1027.9,NaN


In [47]:
df.shape
df.info

<bound method DataFrame.info of             tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2010-01-01   1.6   0.2   3.2   0.0   NaN  324.0  11.8   NaN  1006.1   NaN
2010-01-02   2.7   0.5   5.4   0.0   NaN  254.0  15.0   NaN  1016.0   NaN
2010-01-03   1.9   0.1   3.2   0.0   NaN   43.0   6.4   NaN  1023.6   NaN
2010-01-04  -0.3  -2.0   1.9   0.0   NaN  280.0   8.9   NaN  1020.0   NaN
2010-01-05   1.0  -1.8   3.2   0.3  20.0  178.0   9.8   NaN  1005.9   NaN
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
2024-11-26   5.8   4.6   7.8   0.3   NaN  209.0   9.3  24.1  1015.1   NaN
2024-11-27   5.2   1.2   8.3  16.4   NaN  347.0  15.1  38.9  1013.3   NaN
2024-11-28   1.7  -1.0   4.5   0.0   NaN   78.0   6.5  20.4  1027.9   NaN
2024-11-29   5.3   3.1   8.0   0.0   NaN  151.0  14.3  27.8  1026.3   NaN
2024-11-30  10.0   6.8  11.8   0.0   NaN  178.0  11.7  24.1  1023.1   NaN

[5448

In [48]:
df.apply(pd.isnull).sum()

tavg       0
tmin       1
tmax       1
prcp      21
snow    5336
wdir       3
wspd       2
wpgt    3204
pres       5
tsun    5448
dtype: int64

In [49]:
#Tỷ lệ % các giá trị null trong từng cột
missing_percentage = df.apply(pd.isnull).sum()/df.shape[0] * 100
missing_percentage

tavg      0.000000
tmin      0.018355
tmax      0.018355
prcp      0.385463
snow     97.944200
wdir      0.055066
wspd      0.036711
wpgt     58.810573
pres      0.091777
tsun    100.000000
dtype: float64

In [50]:
# Nhận thấy 3 cột snow, wpgt,tsun toàn giá trị null nên ta sẽ bỏ các cột đó 

columns_to_drop = ['snow', 'wpgt','tsun']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])


In [51]:
df

,tavg,tmin,tmax,prcp,wdir,wspd,pres
time,,,,,,,
2010-01-01,1.6,0.2,3.2,0.0,324.0,11.8,1006.1
2010-01-02,2.7,0.5,5.4,0.0,254.0,15.0,1016.0
2010-01-03,1.9,0.1,3.2,0.0,43.0,6.4,1023.6
2010-01-04,-0.3,-2.0,1.9,0.0,280.0,8.9,1020.0
2010-01-05,1.0,-1.8,3.2,0.3,178.0,9.8,1005.9
...,...,...,...,...,...,...,...
2024-11-26,5.8,4.6,7.8,0.3,209.0,9.3,1015.1
2024-11-27,5.2,1.2,8.3,16.4,347.0,15.1,1013.3
2024-11-28,1.7,-1.0,4.5,0.0,78.0,6.5,1027.9


In [52]:
# Loại bỏ các giá trị ngoại lai dựa trên phương pháp IQR (Interquartile Range)
numeric_columns = df.select_dtypes(include=['float64']).columns
# Hàm loại bỏ giá trị ngoại lai
def remove_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)  # Quartile 1
        Q3 = df[col].quantile(0.75)  # Quartile 3
        IQR = Q3 - Q1                # Interquartile Range
        lower_bound = Q1 - 1.5 * IQR # Giới hạn dưới
        upper_bound = Q3 + 1.5 * IQR # Giới hạn trên
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df
# Áp dụng hàm để loại bỏ giá trị ngoại lai cho các cột số liệu
df = remove_outliers(df, numeric_columns)

In [53]:
# Kiểm tra số dòng trước và sau khi loại bỏ ngoại lai
original_count = len(df)
new_count = len(df)

original_count, new_count

(4552, 4552)

In [54]:
df = df.interpolate(method='linear', limit_direction='forward', axis=0)

In [55]:
df

,tavg,tmin,tmax,prcp,wdir,wspd,pres
time,,,,,,,
2010-01-01,1.6,0.2,3.2,0.0,324.0,11.8,1006.1
2010-01-02,2.7,0.5,5.4,0.0,254.0,15.0,1016.0
2010-01-03,1.9,0.1,3.2,0.0,43.0,6.4,1023.6
2010-01-04,-0.3,-2.0,1.9,0.0,280.0,8.9,1020.0
2010-01-05,1.0,-1.8,3.2,0.3,178.0,9.8,1005.9
...,...,...,...,...,...,...,...
2024-11-25,9.3,6.2,11.9,2.6,230.0,17.5,1001.3
2024-11-26,5.8,4.6,7.8,0.3,209.0,9.3,1015.1
2024-11-28,1.7,-1.0,4.5,0.0,78.0,6.5,1027.9


In [56]:
# Function to classify weather with multiple tags
def classify_weather(row):
    weather_classes = []
    
    if row['prcp'] > 0:
        weather_classes.append('Rainy')
    if row['wspd'] > 10:
        weather_classes.append('Windy')
    if row['tavg'] > 30:
        weather_classes.append('Hot')
    elif 20 <= row['tavg'] <= 30:
        weather_classes.append('Warm')
    elif 10 <= row['tavg'] < 20:
        weather_classes.append('Cool')
    else:
        weather_classes.append('Cold')
    
    return ', '.join(weather_classes)  # Return all classifications as a comma-separated string

# Apply the classification function to each row and create a new 'weather_class' column
df['weather_class'] = df.apply(classify_weather, axis=1)

In [57]:
df

,tavg,tmin,tmax,prcp,wdir,wspd,pres,weather_class
time,,,,,,,,
2010-01-01,1.6,0.2,3.2,0.0,324.0,11.8,1006.1,"Windy, Cold"
2010-01-02,2.7,0.5,5.4,0.0,254.0,15.0,1016.0,"Windy, Cold"
2010-01-03,1.9,0.1,3.2,0.0,43.0,6.4,1023.6,Cold
2010-01-04,-0.3,-2.0,1.9,0.0,280.0,8.9,1020.0,Cold
2010-01-05,1.0,-1.8,3.2,0.3,178.0,9.8,1005.9,"Rainy, Cold"
...,...,...,...,...,...,...,...,...
2024-11-25,9.3,6.2,11.9,2.6,230.0,17.5,1001.3,"Rainy, Windy, Cold"
2024-11-26,5.8,4.6,7.8,0.3,209.0,9.3,1015.1,"Rainy, Cold"
2024-11-28,1.7,-1.0,4.5,0.0,78.0,6.5,1027.9,Cold


In [61]:

from sklearn.linear_model import LinearRegression



# Feature Engineering: Create lagged features for tavg
def create_lagged_features(df, target, lags):
    X, y = [], []
    for i in range(lags, len(df)):
        X.append(df[target].iloc[i-lags:i].values.flatten())
        y.append(df[target].iloc[i])
    return np.array(X), np.array(y)

lags = 3  # Use past 3 days as features
X, y = create_lagged_features(df, target='tavg', lags=lags)

# Train-test split
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Train Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict next 7 days
last_sequence = df['tavg'].iloc[-lags:].values.flatten()
predictions = []
for _ in range(7):
    next_pred = model.predict(last_sequence.reshape(1, -1))[0]
    predictions.append(next_pred)
    # Update sequence with the new prediction
    last_sequence = np.roll(last_sequence, -1)
    last_sequence[-1] = next_pred

# Create a DataFrame for predictions
future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=7)
future_data = pd.DataFrame({
    'avg_temp': predictions  # Rename tavg to avg_temp
}, index=future_dates)



# Classify the predicted weather
future_data['weather_class'] = future_data.apply(classify_weather, axis=1)

# Display the result
print(future_data)


            avg_temp weather_class
2024-12-01  9.388615          Cold
2024-12-02  8.726499          Cold
2024-12-03  9.017284          Cold
2024-12-04  9.282306          Cold
2024-12-05  9.369358          Cold
2024-12-06  9.459849          Cold
2024-12-07  9.576221          Cold
